# class -> OAS
> Automatically generate OAS from python class implementation via OpenAI API
https://chatgpt.com/share/67210fb1-539c-8002-b6ba-b17e72e0a8bf

In [ ]:
#| default_exp ninjalabo.llmcam.class_to_oas

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
import json
from openai import OpenAI
import yaml
from openapi_spec_validator import validate_spec

In [ ]:
class Item:
    def __init__(self, name: str, price: float, quantity: int):
        self.name = name
        self.price = price
        self.quantity = quantity

    def get_total_price(self) -> float:
        return self.price * self.quantity

In [ ]:
item = Item("shoes", 49.0, 2)
item.get_total_price()

98.0

In [ ]:
#| export
#| eval: false
api = OpenAI()

In [ ]:
#|export
content = """
Given the following Python class, generate an OpenAPI 3.0 specification that defines endpoints
for creating, updating, and retrieving instances of the class. Also, define the schema for the class in OpenAPI.
Please return OpenAPI 3.0 specification yaml file only.

Python Class:
class Item:
    def __init__(self, name: str, price: float, quantity: int):
        self.name = name
        self.price = price
        self.quantity = quantity

    def get_total_price(self) -> float:
        return self.price * self.quantity
        """

In [ ]:
#| eval: false
res = api.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an assistant that generates OpenAPI specs."},
        {"role": "user", "content": content}
    ]
)

res = json.loads(res.json())['choices'][0]['message']['content']
res = res.replace("```yaml", "").replace("```", "").strip()

In [ ]:
#print(res)

In [ ]:
# !pip install -qq pyyaml openapi-spec-validator

In [ ]:
def validate_yaml(yaml_buffer):
    try:
        yaml_content = yaml.safe_load(yaml_buffer)
        print("YAML syntax is valid.")
        return yaml_content
    except yaml.YAMLError as e:
        print("YAML syntax error:", e)
        return None

def validate_oas(yaml_content):
    try:
        validate_spec(yaml_content)
        print("OpenAPI specification format is valid.")
    except Exception as e:
        print("OpenAPI validation error:", e)

In [ ]:
#| eval: false
yaml_content = validate_yaml(res)
if yaml_content:
    validate_oas(yaml_content)

YAML syntax is valid.
OpenAPI specification format is valid.


In [ ]:
#| eval: false
with open("oas.yaml", "w") as f:
    yaml.dump(yaml_content, f)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()